In [ ]:
import time
import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_dataset = datasets.FashionMNIST(root='data', train=True, download=True, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]))
test_dataset = datasets.FashionMNIST(root='data', train=False, download=True, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]))

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=2) 
test_loader = DataLoader(test_dataset, batch_size=10, num_workers=2)


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw
Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
print(len(train_dataset))
print(len(test_dataset))

60000
10000


In [ ]:
model = models.vgg16(pretrained=True)
model.features[0] = nn.Conv2d(1, 64, 3, 1, 1)
model.classifier[-1] = nn.Linear(4096, 1000)
model.classifier.add_module('7', nn.ReLU())
model.classifier.add_module('8', nn.Dropout(p=0.5, inplace=False))
model.classifier.add_module('9', nn.Linear(1000, 10))

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [ ]:
for param in model.features[1:].parameters():
  param.requires_grad = False

if torch.cuda.is_available():
    model.to('cuda')
    print(torch.cuda.is_available())

True


In [ ]:
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss()
epochs = 3
losses = []

In [ ]:
for i in range(epochs):
  for j,(images,targets) in enumerate(tqdm.notebook.tqdm(train_loader)):
    if torch.cuda.is_available():
      images, targets = images.to('cuda'), targets.to('cuda')
    
    y_pred = model(images)

    loss = criterion(y_pred,targets.reshape(-1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
 

In [ ]:
total = 0
  
for x_test, y_test in test_loader:
  if torch.cuda.is_available():
    x_test, y_test = x_test.to('cuda'), y_test.to('cuda')
  
  y_pred = model(x_test).argmax(dim=1)
  total += y_pred.eq(y_test).sum()

print("Accuracy is: ", (total/10000.).item()*100, "%")

Accuracy is:  88.84999752044678 %
